[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MatteoFalcioni/Learning-LangGraph/blob/main/notebooks/7_memory.ipynb)

### Setup

#### Install requirements

In [ ]:
%pip install -q -U -r https://raw.githubusercontent.com/MatteoFalcioni/Learning-LangGraph/main/requirements.txt

Note: you may need to restart the kernel to use updated packages.


#### local (notebooks or files)

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # load api keys

True

#### Colab

In [ ]:
from google.colab import userdata
import os

REQUIRED_KEYS = [
    'OPENAI_API_KEY',
    'LANGSMITH_TRACING',
    'LANGSMITH_ENDPOINT',
    'LANGSMITH_API_KEY',
    'LANGSMITH_PROJECT'
]

def _set_colab_keys(key : str):
    # Retrieve the secret value using its key/name
    secret_value = userdata.get(key)
    # set it as a standard OS environment variable
    os.environ[key] = secret_value

for key in REQUIRED_KEYS:
    _set_colab_keys(key)

# Persistence

LangGraph has a built-in persistence layer, implemented through checkpointers. When you compile a graph with a checkpointer, the checkpointer saves a `checkpoint` of the graph state at every step. 

Those checkpoints are saved to a `thread` (a unique identifier) which can be accessed after graph execution. 

Because threads allow access to graph’s state after execution, several powerful capabilities including human-in-the-loop, memory, time travel, and fault-tolerance are all possible.

---

## Memory

**The main persistence feature we want to have a look at is agents' memory**: its implementations is really straightforward. 

LangGraph provides a memory checkpointer out of the bat, the `InMemorySaver` object:

```python
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
```

In order to use this object as our checkpointer, we need to pass it as an argument when we compile our graph: 

```python
graph = builder.compile(checkpointer=checkpointer)
```

And then we need to specify a thread-id in the config dictionary at invocation, i.e. : 

```python
config = {"configurable": {"thread_id": "1"}}
graph.invoke(init_state, config)
```

The coice of the thread id is completely arbitrary, I chose "1" just for simplicity.

Let's see an actual example with a simple graph:

In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.types import Command
from operator import add

class MyState(TypedDict):
    foo: Annotated[list[str], add]

def node_a(state: MyState) -> Command[Literal["node_b"]]:
    return Command(
        update={
            "foo" : ["a"]
        },
        goto="node_b"
    )

def node_b(state: MyState) -> Command[Literal["__end__"]]:
    return Command(
        update={
            "foo" : ["b"]
        },
        goto="__end__"
    )

builder = StateGraph(MyState)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)
builder.add_edge(START, "node_a")
# no edges a->b->END (using Command!)

# instantiate the checkpointer
checkpointer = InMemorySaver()
# compile the graph w/ the checkpointer
graph = builder.compile(checkpointer=checkpointer)

# invoke the graph with a config dict
init_state = {"foo": []}
config = {"configurable": {"thread_id": "1"}}

graph.invoke(init_state, config)

{'foo': ['a', 'b']}

Now that we are using a checkpointer, LangGraph automatically saves state checkpoints of our graph during execution. 

In this graph run, we expect to see 4 checkpoints: 

- Empty checkpoint with `START` as the next node to be executed
- Checkpoint with the user input `{'foo': []}` and `node_a` as the next node to be executed
- Checkpoint with the outputs of `node_a` `{'foo': ['a']}` and `node_b` as the next node to be executed
- Checkpoint with the outputs of `node_b` `{'foo': ['a', 'b']}` and no next nodes to be executed

We can view the last state of the graph by calling `graph.get_state(config)` : this will return a `StateSnapshot` object corresponding to the latest checkpoint:

In [2]:
# get the latest state snapshot
graph.get_state(config)

StateSnapshot(values={'foo': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d99c7-04cc-6017-8002-de12a6228bac'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-15T09:57:17.068898+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d99c7-04ca-60aa-8001-d6eff0508203'}}, tasks=(), interrupts=())


You can get the full history of the graph execution for a given thread by calling `graph.get_state_history(config)`. 

This will return a list of `StateSnapshot` objects associated with the thread ID provided in the config. 

Importantly, the checkpoints will be ordered chronologically with the most recent checkpoint / `StateSnapshot` being the first in the list.

In [3]:
history = graph.get_state_history(config)

for checkpoint in history:
    print(checkpoint)

StateSnapshot(values={'foo': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d99c7-04cc-6017-8002-de12a6228bac'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-15T09:57:17.068898+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d99c7-04ca-60aa-8001-d6eff0508203'}}, tasks=(), interrupts=())
StateSnapshot(values={'foo': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d99c7-04ca-60aa-8001-d6eff0508203'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-12-15T09:57:17.068094+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0d99c7-04c7-66be-8000-3ff5fb122b6f'}}, tasks=(PregelTask(id='1f65d63b-6d7d-7ed9-2d97-a9cbd57411a3', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts=(), state=None, result={'f

It’s also possible to play-back a prior graph execution. If we invoke a graph with a `thread_id` and a `checkpoint_id`, then we will re-play the previously executed steps before a checkpoint that corresponds to the `checkpoint_id`, and only execute the steps after the checkpoint.

In [ ]:
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f0d99c7-04ca-60aa-8001-d6eff0508203"}}  # took this checkpoint id from the above output
graph.invoke(None, config=config)  # execute from that checkpoint on

{'foo': ['a', 'b']}

This type of jumping back to a node, replaying execution or eventually modifying the graph state and continuing  from a given checkpoint, is reffered to as "time travel" in LangGraph. 

If you want to know more, see [notebook 9 - second part](9_human_in_the_loop.ipynb), or check this [LangChain time travel guide](https://docs.langchain.com/oss/python/langgraph/use-time-travel). 

## Agents With Memory

In [ ]:
from typing import Annotated, Literal
from langchain_core.messages import ToolMessage
from langchain.tools import tool, ToolRuntime
from langgraph.graph import StateGraph, START, END
from langchain.agents import AgentState, create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI
from langgraph.types import Command

def reduce_str(left : str | None, right : str | None) -> str:
    """Reduce two strings by replacing."""
    if left is None:
        return ""
    if right is None:
        return ""
    return right

class MyState(AgentState):
    user_id: Annotated[str, reduce_str]

@tool
def get_user_info(runtime: ToolRuntime) -> Command:
    """Look up user info."""
    user_id = runtime.state["user_id"]  
    result_string = "User is Matteo" if user_id == "user_123" else "Unknown user"
    return Command(
        update = {
            "messages" : [ToolMessage(content=result_string, tool_call_id=runtime.tool_call_id)]
        }
    )

agent = create_agent(
    model=ChatOpenAI(model="gpt-4o", temperature=0),
    tools=[get_user_info],
    system_prompt="You are a helpful assistant that provides user information based on user ID.",
    state_schema=MyState,
)

def agent_node(state : MyState) -> Command[Literal["__end__"]]:
    
    result = agent.invoke(state)
    messages = result["messages"]

    return Command(
        update={
            "messages": messages
        },
        goto="__end__"
    )

builder = StateGraph(MyState)
builder.add_node("agent_node", agent_node)
builder.add_edge(START, "agent_node")

checkpointer = InMemorySaver()

graph = builder.compile(checkpointer=checkpointer)

In [ ]:
from langchain_core.messages import HumanMessage

init_state = {"messages" : [HumanMessage(content="Get user info")], "user_id": "user_123"}
config = {"configurable": {"thread_id": "test_123"}}

result = graph.invoke(init_state, config=config)

In [ ]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Get user info
================================== Ai Message ==================================
Tool Calls:
  get_user_info (call_rhoOYHGnsOSj4TgNcP85p4PI)
 Call ID: call_rhoOYHGnsOSj4TgNcP85p4PI
  Args:
================================= Tool Message =================================
Name: get_user_info

User is Matteo
================================== Ai Message ==================================

The user is Matteo.


In [ ]:
second_message = HumanMessage(content="What was my first question? ANd what was your answer?")
new_state = {"messages" : [second_message], "user_id": ""}

for chunk in graph.stream(new_state, config=config):  # same config as before (same thread)
    
    for node_name, values in chunk.items():
        if 'messages' in values:
            values['messages'][-1].pretty_print()

================================== Ai Message ==================================

Your first question was "Get user info," and my answer was "The user is Matteo."


---

Another very useful concept I recommend you to have a look at is memory stores. 

The idea behind this is: "*what if we want to retain some information across threads? Consider the case of a chatbot where we want to retain specific information about the user across **all** chat conversations (e.g., threads) with that user...*"

You can imagine how useful this can be - all the Chatbots you find online have this feature in one way or another. 

We will not go through the details of this implementation, but you can find details here: [Memory Store](https://docs.langchain.com/oss/python/langgraph/persistence#memory-store).